# Feedback Loop in RAG

이 노트북에서는 시간이 지날수록 스스로 발전하는 피드백 루프 기반의 RAG 시스템을 구현해보겠습니다. 사용자의 피드백을 받아들이고 이를 바탕으로 개선해나가면서, 시스템은 매번 더 정확하고 유용한 답변을 제시하는 법을 배워갑니다.

기존의 RAG 시스템은 주로 임베딩 유사도에 따라 정보를 검색하는 방식으로, 변화에 둔감한 면이 있었습니다. 하지만 피드백 루프를 도입하면 시스템은 다음과 같은 방식으로 발전됩니다.
1. 잘 작동했던 방식과 그렇지 않았던 방식을 기억하고
2. 시간이 지날수록 문서의 중요도를 조정하며
3. 효과적인 질문과 답변 쌍을 지식으로 쌓고
4. 사용자와의 상호작용을 통해 점점 더 똑똑해집니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv
from datetime import datetime

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## 추출된 텍스트 청크 분할
텍스트를 추출한 뒤에는 검색 정확도를 높이기 위해 조금씩 겹치도록 나눠서 작은 단위로 분할(chunk)합니다.  

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 일부가 겹치도록 chunking 합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 리스트
    
    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])

    return chunks  

## Simple Vector Store 구축
NumPy를 사용하여 간단한 Vecotr store 구축

In [ ]:
class SimpleVectorStore:
    """
    NumPy를 사용하여 간단한 Vecotr store 구축
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목 추가

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        쿼리 임베딩과 가장 비슷한 항목들을 코사인 유사도를 이용해 찾는 함수

        Args:
            query_embedding (List[float]): 쿼리 임베딩 벡터.
            k (int): 반환할 결과의 수.
            filter_func (callable, optional): 메타데이터를 기반으로 결과를 필터링하는 함수.
                                            메타데이터 딕셔너리를 입력으로 받아 boolean을 반환.

        Returns:
            List[Dict]: 상위 k개 유사 항목, 각 항목은 다음을 포함:
                - text: 원본 텍스트
                - metadata: 연관된 메타데이터
                - similarity: 코사인 유사도 점수
                - relevance_score: 메타데이터 기반 관련성 또는 계산된 유사도
                
        Note: 벡터가 저장되어 있지 않거나 필터를 통과하지 못하면 빈 리스트를 반환.
        """
        if not self.vectors:
            return []  # 벡터 저장소가 비어있으면 빈 리스트 반환
        
        # 쿼리 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 쿼리와 각 저장된 벡터 사이의 코사인 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 필터 기준을 통과하지 못하는 항목 건너뛰기
            if filter_func and not filter_func(self.metadata[i]):
                continue
                
            # 코사인 유사도 계산: dot product / (norm1 * norm2)
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 인덱스와 유사도 점수 저장
        
        # 유사도 점수에 따라 정렬 (내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 딕셔너리 생성
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score,
                # 메타데이터에서 이미 존재하는 관련성 점수가 있으면 사용, 없으면 유사도 점수 사용
                "relevance_score": self.metadata[idx].get("relevance_score", score)
            })
        
        return results

## 임베딩 생성

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

## Feedback System Functions
이제 피드백 시스템의 핵심 요소들을 구현해보겠습니다.

In [ ]:
def get_user_feedback(query, response, relevance, quality, comments=""):
    """
    사용자 피드백을 보기 쉽게 딕셔너리 형태로 정리합니다.
    
    Args:
        query (str): 사용자 쿼리
        response (str): 시스템 응답
        relevance (int): 관련성 점수 (1-5)
        quality (int): 품질 점수 (1-5)
        comments (str): 선택적 피드백 코멘트
        
    Returns:
        Dict: 사용자 피드백
    """
    return {
        "query": query,
        "response": response,
        "relevance": int(relevance),
        "quality": int(quality),
        "comments": comments,
        "timestamp": datetime.now().isoformat()
    }

In [ ]:
import json
def store_feedback(feedback, feedback_file="feedback_data.json"):
    """
    피드백을 JSON 파일에 저장
    
    Args:
        feedback (Dict): 피드백 데이터
        feedback_file (str): 피드백 파일 경로
    """
    with open(feedback_file, "a") as f:
        json.dump(feedback, f)
        f.write("\n")

In [ ]:
import json
def load_feedback_data(feedback_file="feedback_data.json"):
    """
    피드백 데이터를 파일에서 로드
    
    Args:
        feedback_file (str): feedback file 경로
        
    Returns:
        List[Dict]: 피드백 리스트
    """
    feedback_data = []
    try:
        with open(feedback_file, "r") as f:
            for line in f:
                if line.strip():
                    feedback_data.append(json.loads(line.strip()))
    except FileNotFoundError:
        print("No feedback data file found. Starting with empty feedback.")
    
    return feedback_data

## Document Processing with Feedback Awareness

In [ ]:
def process_document(file_path, chunk_size=1000, chunk_overlap=200):
    """
    RAG를 위한 문서를 처리하고, 피드백 루프까지 포함하는 전체 파이프라인
    이 함수는 다음과 같은 과정을 순차적으로 수행합니다:
    1. PDF 파일에서 텍스트 추출
    2. 텍스트를 일정 단위로 나누되, 중간에 겹치는 부분을 포함
    3. 각 조각에 대한 임베딩 생성
    4. 메타데이터와 함께 벡터 데이터베이스에 저장

    Args:
    file_path (str): 파일 경로
    chunk_size (int): 각 텍스트 청크의 크기(문자)
    chunk_overlap (int): 연속된 청크 간 중복 범위(문자)

    Returns:
    Tuple[List[str], SimpleVectorStore]: 튜플에 포함된 요소:
        - 문서 청크 리스트
        - 임베딩과 메타데이터가 포함된 벡터 저장소
    """
    # 1. PDF에서 텍스트 추출
    #print("Extracting text from PDF...")
    #extracted_text = extract_text_from_pdf(pdf_path)

    # 1. 텍스트 파일 로드
    extracted_text = load_text_file(file_path)
    
    # 2. 청크로 분할
    print("Chunking text...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(chunks)} text chunks")
    
    # 3. 각 청크에 대한 임베딩 생성
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings(embedding_model, chunks, device=device, batch_size=1)
    
    # 4. 벡터 저장소 초기화
    store = SimpleVectorStore()
    
    # 5. 각 청크와 임베딩을 벡터 저장소에 추가
    # 피드백 기반 개선을 위한 메타데이터 포함
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={
                "index": i,                # 원본 문서에서의 위치
                "source": file_path,        # 원본 문서 경로
                "relevance_score": 1.0,    # 초기 relevance score (피드백에 따라 업데이트됨)
                "feedback_count": 0        # 이 청크에 대해 받은 피드백 수
            }
        )
    
    print(f"Added {len(chunks)} chunks to the vector store")
    return chunks, store

## Relevance Adjustment Based on Feedback

In [ ]:
def assess_feedback_relevance(query, doc_text, feedback):
    """
    LLM을 활용해 이전 피드백이 현재 쿼리와 문서에 얼마나 관련 있는지를 판단합니다.
    
    이 함수는 현재 쿼리와 문서 내용, 그리고 과거의 쿼리 및 피드백을 함께 LLM에 전달하여,
    어떤 피드백이 이번 검색 결과에 영향을 줄 수 있을지를 평가합니다.
    
    Args:
        query (str): 현재 사용자 쿼리
        doc_text (str): 평가할 문서의 텍스트 내용
        feedback (Dict): 'query'와 'response' 키가 포함된 이전 피드백 데이터
        
    Returns:
        bool: 피드백이 현재 쿼리와 문서와 관련 있는지 여부
    """
    # 시스템 프롬프트
    system_prompt = """You are an AI system that determines if a past feedback is relevant to a current query and document.
    Answer with ONLY 'yes' or 'no'. Your job is strictly to determine relevance, not to provide explanations."""

    # 사용자 프롬프트
    user_prompt = f"""
    Current query: {query}
    Past query that received feedback: {feedback['query']}
    Document content: {doc_text[:500]}... [truncated]
    Past response that received feedback: {feedback['response'][:500]}... [truncated]

    Is this past feedback relevant to the current query and document? (yes/no)
    """

    # 평가
    response = client_openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  
    )
    
    
    answer = response.choices[0].message.content.strip().lower()
    return 'yes' in answer  # 'yes'가 포함되어 있으면 True 반환

In [ ]:
def adjust_relevance_scores(query, results, feedback_data):
    """
    과거 피드백을 반영해 문서의 연관성 점수를 조정하고, 검색 결과의 품질을 높입니다.

    이 함수는 이전 사용자 피드백을 분석해, 현재 쿼리와 관련된 내용을 선별합니다.
    그 후 피드백의 평가 기준에 따라 점수 보정값을 계산하고, 이를 기반으로 검색 결과의 순위를 다시 매깁니다.
    
    Args:
        query (str): 현재 사용자 쿼리
        results (List[Dict]): 유사도 점수가 포함된 검색된 문서 리스트
        feedback_data (List[Dict]): 사용자 평가가 포함된 과거 피드백 리스트
        
    Returns:
        List[Dict]: 관련성 점수가 조정된 결과, 새로운 점수에 따라 정렬됨
    """
    # 피드백 데이터가 없으면 원본 결과 반환
    if not feedback_data:
        return results
    
    print("Adjusting relevance scores based on feedback history...")
    
    # 각 검색된 문서에 대해 처리
    for i, result in enumerate(results):
        document_text = result["text"]
        relevant_feedback = []
        
        # 해당 문서와 쿼리 조합에 대해,
        # 각 과거 피드백이 얼마나 관련 있는지를 LLM을 통해 평가하고 관련 피드백을 찾아냅니다.
        for feedback in feedback_data:
            is_relevant = assess_feedback_relevance(query, document_text, feedback)
            if is_relevant:
                relevant_feedback.append(feedback)
        
        # 관련성 있는 피드백이 있으면 점수 조정
        if relevant_feedback:
            # 모든 적용 가능한 피드백 항목에서 평균 관련성 등급 계산
            # 피드백 관련성은 1~5 범위 (1:관련 없음, 5:매우 관련 있음)
            avg_relevance = sum(f['relevance'] for f in relevant_feedback) / len(relevant_feedback)
            
            # 평균 관련성을 0.5-1.5 범위의 점수 수정자로 변환
            # - 3/5 미만의 점수는 원래 유사도를 감소시킴 (modifier < 1.0)
            # - 3/5 이상의 점수는 원래 유사도를 증가시킴 (modifier > 1.0)
            modifier = 0.5 + (avg_relevance / 5.0)
            
            # 원래 유사도에 modifier 적용
            original_score = result["similarity"]
            adjusted_score = original_score * modifier
            
            # 결과 딕셔너리에 새로운 점수와 피드백 메타데이터 업데이트
            result["original_similarity"] = original_score  # 원래 유사도 유지
            result["similarity"] = adjusted_score           # 기본 점수 업데이트
            result["relevance_score"] = adjusted_score      # 관련성 점수 업데이트
            result["feedback_applied"] = True               # 피드백 적용 표시
            result["feedback_count"] = len(relevant_feedback)  # 사용된 피드백 항목 수
            
            # 유사도 조정에 대한 세부 정보 logging
            print(f"  Document {i+1}: Adjusted score from {original_score:.4f} to {adjusted_score:.4f} based on {len(relevant_feedback)} feedback(s)")
    
    # 조정된 유사도 점수에 따라 결과 다시 정렬
    results.sort(key=lambda x: x["similarity"], reverse=True)
    
    return results

## Fine-tuning Our Index with Feedback

In [ ]:
def fine_tune_index(current_store, chunks, feedback_data):
    """
    시간이 지남에 따라 검색 품질을 높일 수 있도록, 우수한 피드백을 벡터 스토어에 반영합니다.
    
    이 함수는 다음과 같은 방식으로 지속적인 학습을 수행합니다:
    1. 높은 평가를 받은 Q&A 쌍 등, 품질이 검증된 피드백을 선별
    2. 성공적인 상호작용에서 새로운 검색 항목을 생성
    3. 이 항목들을 강화된 가중치와 함께 벡터 스토어에 추가
    
    Args:
        current_store (SimpleVectorStore): 원본 문서 청크가 포함된 현재 벡터 저장소
        chunks (List[str]): 원본 문서 텍스트 청크
        feedback_data (List[Dict]): 관련성과 품질 등급이 포함된 과거 사용자 피드백
        
    Returns:
        SimpleVectorStore: 원본 청크와 피드백 기반 콘텐츠가 포함된 개선된 벡터 저장소
    """
    print("Fine-tuning index with high-quality feedback...")
    
    # 연관도와 응답 품질이 모두 4점 이상인, 우수한 응답만 선별합니다.
    # 가장 성공적인 상호작용만을 학습에 반영하기 위한 과정입니다.
    good_feedback = [f for f in feedback_data if f['relevance'] >= 4 and f['quality'] >= 4]
    
    if not good_feedback:
        print("No high-quality feedback found for fine-tuning.")
        return current_store  # 좋은 피드백이 없으면 원본 저장소 반환
    
    # 원본 청크와 피드백 기반 콘텐츠가 포함된 새로운 저장소 초기화
    new_store = SimpleVectorStore()
    
    # 먼저 원본 문서 청크와 기존 메타데이터를 전송
    for i in range(len(current_store.texts)):
        new_store.add_item(
            text=current_store.texts[i],
            embedding=current_store.vectors[i],
            metadata=current_store.metadata[i].copy()  # 참조 문제를 피하기 위해 복사본을 사용합니다.
        )
    
    # 좋은 피드백에서 향상된 콘텐츠 생성 및 추가
    for feedback in good_feedback:
        # 질문과 고품질 답변을 결합하는 새로운 문서 포맷팅
        # 이는 사용자 쿼리에 직접 주소를 지정하는 검색 가능한 콘텐츠를 생성합니다
        enhanced_text = f"Question: {feedback['query']}\nAnswer: {feedback['response']}"
        
        # 이 새로운 문서에 대한 임베딩 벡터 생성
        embedding = create_embeddings(embedding_model, chunks, device=device, batch_size=1)[0]
        
        # 원본 출처와 중요성을 식별하는 특수 메타데이터로 벡터 저장소에 추가
        new_store.add_item(
            text=enhanced_text,
            embedding=embedding,
            metadata={
                "type": "feedback_enhanced",  # 피드백에서 파생됨
                "query": feedback["query"],   # 원본 쿼리 저장
                "relevance_score": 1.2,       # 초기 관련성 증가
                "feedback_count": 1,          # 피드백 통합 추적
                "original_feedback": feedback # 완전한 피드백 레코드 유지
            }
        )
        
        print(f"Added enhanced content from feedback: {feedback['query'][:50]}...")
    
    # 개선에 대한 요약 통계 logging
    print(f"Fine-tuned index now has {len(new_store.texts)} items (original: {len(chunks)})")
    return new_store

## Complete RAG Pipeline with Feedback Loop

In [ ]:
def generate_response(query, context, model_name='gpt-4.1-nano'):

    # 시스템 프롬프트
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
    
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        Please answer the question based only on the context provided above. Be concise and accurate.
    """
    
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content

In [ ]:
def rag_with_feedback_loop(query, vector_store, feedback_data, k=5, model_name="gpt-4.1-mini"):
    """
    피드백 루프를 포함한 RAG 파이프라인을 구현합니다.    
    
    Args:
        query (str): 사용자 쿼리
        vector_store (SimpleVectorStore): 문서 청크가 포함된 벡터 저장소
        feedback_data (List[Dict]): 피드백 기록
        k (int): 검색할 문서 수
        model (str): 응답 생성을 위한 LLM 모델
        
    Returns:
        Dict: 쿼리, 검색된 문서, 응답이 포함된 결과
    """
    print(f"\n=== Processing query with feedback-enhanced RAG ===")
    print(f"Query: {query}")
    
    # 1. 쿼리 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)[0]
    
    # 2. 쿼리 임베딩을 기반으로 초기 검색 수행
    results = vector_store.similarity_search(query_embedding, k=k)
    
    # 3. 피드백을 기반으로 검색된 문서의 관련성 점수 조정
    adjusted_results = adjust_relevance_scores(query, results, feedback_data)
    
    # 4. 조정된 결과에서 텍스트 추출하여 컨텍스트 생성
    retrieved_texts = [result["text"] for result in adjusted_results]
    
    # 5. 검색된 텍스트를 연결하여 응답 생성을 위한 컨텍스트 생성
    context = "\n\n---\n\n".join(retrieved_texts)
    
    # 6. 컨텍스트와 쿼리를 사용하여 응답 생성
    print("Generating response...")
    response = generate_response(query, context, model_name)
    
    # 7. 최종 결과 컴파일
    result = {
        "query": query,
        "retrieved_documents": adjusted_results,
        "response": response
    }
    
    print("\n=== Response ===")
    print(response)
    
    return result

## Complete Workflow: From Initial Setup to Feedback Collection

In [ ]:
def full_rag_workflow(file_path, query, feedback_data=None, feedback_file="feedback_data.json", fine_tune=False):
    """
    피드백을 통합해 지속적으로 개선되는 RAG 워크플로우 전체를 실행합니다.
    이 함수는 Retrieval-Augmented Generation 과정을 다음 단계에 따라 종합적으로 수행합니다:

    1. 기존 피드백 데이터 불러오기
    2. 문서를 처리하고 chunk 단위로 분할
    3. 이전 피드백을 반영해 벡터 인덱스를 선택적으로 보정
    4. 피드백 기반 연관성 점수를 활용해 검색 및 생성 수행
    5. 새로운 사용자 피드백 수집
    6. 시스템이 점차 학습할 수 있도록 피드백 저장

    Args:
        file_path (str): 파일 경로
        query (str): 사용자 쿼리
        feedback_data (List[Dict], optional): 이미 로드된 피드백 데이터, None인 경우 파일에서 로드
        feedback_file (str): 피드백 기록을 저장하는 JSON 파일 경로
        fine_tune (bool): 성공적인 과거 Q&A 쌍을 활용하여 인덱스를 개선할지 여부
        
    Returns:
        Dict: 응답과 검색 메타데이터가 포함된 결과
    """
    # 1. 기존 피드백 데이터 불러오기
    if feedback_data is None:
        feedback_data = load_feedback_data(feedback_file)
        print(f"Loaded {len(feedback_data)} feedback entries from {feedback_file}")
    
    # 2. 문서를 처리하고 chunk 단위로 분할
    chunks, vector_store = process_document(file_path)
    
    # 3. 이전 피드백을 반영해 벡터 인덱스를 선택적으로 보정
    if fine_tune and feedback_data:
        vector_store = fine_tune_index(vector_store, chunks, feedback_data)
    
    # 4. 피드백 기반 연관성 점수를 활용해 검색 및 생성 수행
    result = rag_with_feedback_loop(query, vector_store, feedback_data)
    
    # 5. 새로운 사용자 피드백 수집
    print("\n=== Would you like to provide feedback on this response? ===")
    print("Rate relevance (1-5, with 5 being most relevant):")
    relevance = input()
    
    print("Rate quality (1-5, with 5 being highest quality):")
    quality = input()
    
    print("Any comments? (optional, press Enter to skip)")
    comments = input()
    
    # 6. 피드백을 구조화된 데이터로 포맷팅
    feedback = get_user_feedback(
        query=query,
        response=result["response"],
        relevance=int(relevance),
        quality=int(quality),
        comments=comments
    )
    
    # 7. 시스템이 점차 학습할 수 있도록 피드백 저장
    store_feedback(feedback, feedback_file)
    print("Feedback recorded. Thank you!")
    
    return result

## Evaluating Our Feedback Loop

In [ ]:
def evaluate_feedback_loop(pdf_path, test_queries, reference_answers=None):
    """
    피드백 루프가 RAG 성능에 미치는 영향을 평가합니다.
    
    이 함수는 피드백 적용 전후의 응답 결과를 비교하여, 피드백이 검색 및 생성 품질에 어떤 영향을 주는지 확인하는 실험을 수행합니다.
    
    실행 흐름:
    1. 피드백 없이 테스트 쿼리 실행
    2. 기준 답변이 있을 경우, 이를 기반으로 피드백 생성
    3. 생성된 피드백을 활용하여 동일한 쿼리를 다시 실행
    4. 두 결과를 비교하여 피드백 효과를 정량적으로 분석

    Args:
        pdf_path (str): 검색 대상이 되는 PDF 문서 경로
        test_queries (List[str]): 테스트할 질의 리스트
        reference_answers (List[str], optional): 기준 정답. 제공되면 피드백 생성 및 성능 평가에 활용

    Returns:
        Dict: 평가 결과 딕셔너리. 아래 내용을 포함합니다.
            - round1_results: 피드백 없이 실행한 결과
            - round2_results: 피드백을 적용한 결과
            - comparison: 두 결과 간의 비교 분석
    """
    print("=== 피드백 루프 영향 평가 시작 ===")
    
    # 임시 피드백 파일 생성 (이번 평가에서만 사용)
    temp_feedback_file = "temp_evaluation_feedback.json"
    
    # 초기 피드백 데이터 (비어 있음)
    feedback_data = []
    
    # ----------------------- 1차 실행: 피드백 없이 -----------------------
    print("\n=== 1차 실행: 피드백 없음 ===")
    round1_results = []
    
    for i, query in enumerate(test_queries):
        print(f"\nQuery {i+1}: {query}")
        
        # 문서 처리 및 벡터스토어 생성
        chunks, vector_store = process_document(pdf_path)
        
        # 피드백 없이 RAG 실행
        result = rag_with_feedback_loop(query, vector_store, [])
        round1_results.append(result)
        
        # 기준 답변이 있다면, 이를 바탕으로 피드백 생성
        if reference_answers and i < len(reference_answers):
            similarity_to_ref = calculate_similarity(result["response"], reference_answers[i])
            relevance = max(1, min(5, int(similarity_to_ref * 5)))
            quality = max(1, min(5, int(similarity_to_ref * 5)))
            
            feedback = get_user_feedback(
                query=query,
                response=result["response"],
                relevance=relevance,
                quality=quality,
                comments=f"기준 답변과 유사도 기반 피드백: {similarity_to_ref:.2f}"
            )
            
            feedback_data.append(feedback)
            store_feedback(feedback, temp_feedback_file)
    
    # ----------------------- 2차 실행: 피드백 적용 -----------------------
    print("\n=== 2차 실행: 피드백 적용 ===")
    round2_results = []
    
    # 문서를 다시 처리하고, 피드백 기반으로 인덱스를 보완
    chunks, vector_store = process_document(pdf_path)
    vector_store = fine_tune_index(vector_store, chunks, feedback_data)
    
    for i, query in enumerate(test_queries):
        print(f"\nQuery {i+1}: {query}")
        
        result = rag_with_feedback_loop(query, vector_store, feedback_data)
        round2_results.append(result)
    
    # ----------------------- 결과 비교 -----------------------
    comparison = compare_results(test_queries, round1_results, round2_results, reference_answers)
    
    # 임시 피드백 파일 정리
    if os.path.exists(temp_feedback_file):
        os.remove(temp_feedback_file)
    
    return {
        "round1_results": round1_results,
        "round2_results": round2_results,
        "comparison": comparison
    }


## Helper Functions for Evaluation

In [ ]:
def calculate_similarity(text1, text2):
    """
    두 텍스트 간 의미적 유사도를 임베딩을 사용하여 계산합니다.
    
    Args:
        text1 (str): 첫 번째 텍스트
        text2 (str): 두 번째 텍스트
        
    Returns:
        float: 0과 1 사이의 유사도 점수
    """
    # 두 텍스트의 임베딩 생성
    embedding1 = create_embeddings(embedding_model, [text1], device=device, batch_size=1)[0]
    embedding2 = create_embeddings(embedding_model, [text2], device=device, batch_size=1)[0]
    
    # 임베딩을 numpy 배열로 변환
    vec1 = np.array(embedding1)
    vec2 = np.array(embedding2)
    
    # 두 벡터 간 코사인 유사도 계산
    similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    
    return similarity

In [ ]:
def compare_results(queries, round1_results, round2_results, reference_answers=None):
    """
    피드백 적용 전후의 RAG 응답을 비교합니다.
    
    Args:
        queries (List[str]): 테스트에 사용된 질의 리스트
        round1_results (List[Dict]): 피드백 없이 실행한 응답 결과
        round2_results (List[Dict]): 피드백 적용 후의 응답 결과
        reference_answers (List[str], optional): 기준 정답 (있을 경우 비교 분석에 활용)
        
    Returns:
        str: 응답 품질 비교에 대한 분석 결과 목록
    """
    print("\n=== 결과 비교 시작 ===")
    
    # 분석 역할을 부여할 시스템 프롬프트
    system_prompt = """당신은 RAG 시스템 응답을 평가하는 전문가입니다. 다음 두 응답을 비교해주세요:
        1. 일반 RAG: 피드백 없이 수행된 응답
        2. 피드백 강화 RAG: 피드백 루프를 통해 개선된 응답

        아래 기준에 따라 어떤 응답이 더 나은지 분석해 주세요:
        - 질의와의 관련성
        - 정보의 정확성
        - 내용의 충실도
        - 표현의 명확성과 간결함
    """

    comparisons = []
    
    # 각 쿼리에 대해 응답을 비교하고 분석 생성
    for i, (query, r1, r2) in enumerate(zip(queries, round1_results, round2_results)):
        # 비교 대상이 되는 두 응답을 포함한 프롬프트 작성
        comparison_prompt = f"""
        Query: {query}

        일반 RAG 응답:
        {r1["response"]}

        피드백 강화 RAG 응답:
        {r2["response"]}
        """

        # 기준 정답이 있다면 함께 제시
        if reference_answers and i < len(reference_answers):
            comparison_prompt += f"""
            기준 정답:
            {reference_answers[i]}
            """

        # 비교 지시 문장 추가
        comparison_prompt += """
        위 두 응답을 비교하고, 어떤 응답이 더 나은지 그 이유와 함께 설명해주세요.
        특히 피드백 루프가 응답 품질에 어떤 영향을 미쳤는지에 집중해주세요.
        """

        # OpenAI API를 호출하여 분석 요청
        response = client_openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": comparison_prompt}
            ],
            temperature=0
        )
        
        # 분석 결과 저장
        comparisons.append({
            "query": query,
            "analysis": response.choices[0].message.content
        })
        
        # 간략 출력 (200자까지)
        print(f"\nQuery {i+1}: {query}")
        print(f"Analysis: {response.choices[0].message.content[:200]}...")
    
    return comparisons


## Evaluation of the feedback loop (Custom Validation Queries)

In [ ]:
import pandas as pd
# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')

# test queries 정의
test_queries = [
    df['query'][0],
    # df['query'][1],
    # df['query'][2],
    # df['query'][3],
    # df['query'][4],
    # df['query'][5],
    # df['query'][6],
    # df['query'][7]
    # ...
]

# 평가를 위한 기준 답변 정의
reference_answers = [
    df['generation_gt'][0],
    # df['generation_gt'][1],
    # df['generation_gt'][2],
    # df['generation_gt'][3],
    # df['generation_gt'][4],
    # df['generation_gt'][5],
    # df['generation_gt'][6],
    # df['generation_gt'][7],
    # ...
    ]


# 파일 경로
file_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

# 평가 실행
evaluation_results = evaluate_feedback_loop(
    file_path=file_path,
    test_queries=test_queries,
    reference_answers=reference_answers
)

In [ ]:
########################################
# # Run a full RAG workflow
########################################

# # Run an interactive example
# print("\n\n=== INTERACTIVE EXAMPLE ===")
# print("Enter your query about AI:")
# user_query = input()

# # Load accumulated feedback
# all_feedback = load_feedback_data()

# # Run full workflow
# result = full_rag_workflow(
#     pdf_path=pdf_path,
#     query=user_query,
#     feedback_data=all_feedback,
#     fine_tune=True
# )

########################################
# # Run a full RAG workflow
########################################